In [94]:
# token


# Bibliothèques
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
from hubspot import HubSpot


In [68]:
# Connection à l'API
api_client = HubSpot(access_token=token)

# Valeur totale des deals avec et sans pondération

### Récupération des infos de l'entreprise

In [71]:
id=7496989169
# Récupération de toute les propriété d'une entreprise
# api_client.crm.properties.core_api.get_all(object_type="companies")

# properties=["name","city","country","industry","hs_analytics_num_visits","hs_annual_revenue_currency_code","total_money_raised","total_revenue","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","owneremail","numberofemployees","hs_total_deal_value","total_money_raised","hs_predictivecontactscore_v2","hs_target_account_probability","hs_predictivecontactscore_v2_next_max_max_d4e58c1e","num_associated_contacts","num_conversion_events","num_conversion_events_cardinality_sum_d095f14b"]
properties_needed=["hs_predictivecontactscore_v2","valeur_totale_des_deals","valeur_totale_des_deals_ponderee"]
company=api_client.crm.companies.basic_api.get_by_id(id,properties=properties_needed)
company_data=company.to_dict()
company_data["properties"]

{'createdate': '2023-04-28T09:53:40.624Z',
 'hs_lastmodifieddate': '2023-04-28T16:16:00.736Z',
 'hs_object_id': '7496989169',
 'hs_predictivecontactscore_v2': '3.6',
 'valeur_totale_des_deals': '0',
 'valeur_totale_des_deals_ponderee': '0'}

# Récupération des infos des deals

In [98]:
# Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
deals_id=[]
deals=api_client.crm.deals.get_all()
for deal in deals:
    deals_id.append(deal.to_dict()["id"])
deals_id

['7280660186', '7280697021', '7280697041']

In [100]:
# deal1=7280660186
# deal2=7280697021
# deal3=7280697041
# Récupération de toute les propriété d'un deal
# api_client.crm.properties.core_api.get_all(object_type="deals")

#properties=["amount","hs_likelihood_to_close","hs_predicted_amount","hs_priority","hs_closed_amount","hs_deal_stage_probability","hs_deal_stage_probability_shadow","hs_forecast_amount","hs_forecast_probability","hs_is_closed","hs_is_closed_won","hs_projected_amount","num_associated_contacts"]
properties_needed=["amount","hs_deal_stage_probability"]
for id in deals_id :
    deal=api_client.crm.deals.basic_api.get_by_id(id,properties=properties_needed)
    deal_data=deal.to_dict()
    print(f"Deal: {id} ")
    print(deal_data["properties"])

Deal: 7280660186 
{'amount': '25400', 'createdate': '2023-04-28T09:55:04.007Z', 'hs_deal_stage_probability': '0.200000000000000011102230246251565404236316680908203125', 'hs_lastmodifieddate': '2023-04-30T01:31:20.094Z', 'hs_object_id': '7280660186'}
Deal: 7280697021 
{'amount': '300000', 'createdate': '2023-04-28T09:56:12.025Z', 'hs_deal_stage_probability': '0.40000000000000002220446049250313080847263336181640625', 'hs_lastmodifieddate': '2023-04-29T12:14:08.235Z', 'hs_object_id': '7280697021'}
Deal: 7280697041 
{'amount': '80098', 'createdate': '2023-04-28T09:57:05.960Z', 'hs_deal_stage_probability': '0.90000000000000002220446049250313080847263336181640625', 'hs_lastmodifieddate': '2023-04-29T12:13:43.668Z', 'hs_object_id': '7280697041'}


In [101]:
# Récupération du montant total des deals sans pondération
#deals_id=[deal1,deal2,deal3]
montant_deals=0
for id in deals_id:
    deal=api_client.crm.deals.basic_api.get_by_id(id)
    deal_data=deal.to_dict()
    montant_deals+=float(deal_data["properties"]["amount"])
print(montant_deals)


405498.0


In [103]:
# Récupération du montant total des deals avec pondération
montant_deals_pond=0
for id in deals_id:
    deal=api_client.crm.deals.basic_api.get_by_id(id,properties=properties_needed)
    deal_data=deal.to_dict()
    montant_deals_pond+=float(deal_data["properties"]["amount"]) * float(deal_data["properties"]["hs_deal_stage_probability"])
print(montant_deals_pond)

197168.2


In [105]:
# Mise à jour des infos de l'entreprise
comp_id="7496989169"
update_valeurs_deals = SimplePublicObjectInput(
        properties={
                "valeur_totale_des_deals":montant_deals ,
                    "valeur_totale_des_deals_ponderee":montant_deals_pond})
api_client.crm.companies.basic_api.update(simple_public_object_input=update_valeurs_deals,company_id="7496989169")

{'archived': False,
 'archived_at': None,
 'created_at': datetime.datetime(2023, 4, 28, 9, 53, 40, 624000, tzinfo=tzutc()),
 'id': '7496989169',
 'properties': {'createdate': '2023-04-28T09:53:40.624Z',
                'hs_all_owner_ids': '911942887',
                'hs_created_by_user_id': '51189390',
                'hs_lastmodifieddate': '2023-04-30T01:39:28.980Z',
                'hs_object_id': '7496989169',
                'hs_pipeline': 'companies-lifecycle-pipeline',
                'hs_updated_by_user_id': '51189390',
                'hs_user_ids_of_all_owners': '51189390',
                'hubspot_owner_assigneddate': '2023-04-28T09:53:40.624Z',
                'hubspot_owner_id': '911942887',
                'lifecyclestage': 'opportunity',
                'valeur_totale_des_deals': '405498.0',
                'valeur_totale_des_deals_ponderee': '197168.2'},
 'properties_with_history': None,
 'updated_at': datetime.datetime(2023, 4, 30, 1, 39, 28, 980000, tzinfo=tzutc())}

In [106]:
# Récupération des infos après update
id=7496989169
# Récupération de toute les propriété d'une entreprise
# api_client.crm.properties.core_api.get_all(object_type="companies")

# properties=["name","city","country","industry","hs_analytics_num_visits","hs_annual_revenue_currency_code","total_money_raised","total_revenue","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","owneremail","numberofemployees","hs_total_deal_value","total_money_raised","hs_predictivecontactscore_v2","hs_target_account_probability","hs_predictivecontactscore_v2_next_max_max_d4e58c1e","num_associated_contacts","num_conversion_events","num_conversion_events_cardinality_sum_d095f14b,"num_associated_deals"]
properties_needed=["name","city","country","industry","numberofemployees","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","num_associated_deals"]
company=api_client.crm.companies.basic_api.get_by_id(id,properties=properties_needed)
company_data=company.to_dict()
company_data["properties"]

{'city': 'Grenoble',
 'country': 'France',
 'createdate': '2023-04-28T09:53:40.624Z',
 'hs_lastmodifieddate': '2023-04-30T01:39:28.980Z',
 'hs_object_id': '7496989169',
 'industry': 'INFORMATION_TECHNOLOGY_AND_SERVICES',
 'name': 'company',
 'num_associated_deals': '3',
 'numberofemployees': '20',
 'valeur_totale_des_deals': '405498.0',
 'valeur_totale_des_deals_ponderee': '197168.2'}